<a href="https://colab.research.google.com/github/mmsamiei/world/blob/main/world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Faker

In [2]:
from faker import Faker
import random
import datetime
from itertools import filterfalse

In [3]:
fake = Faker()

In [4]:
class Location:

  def __init__(self, map_size=100):
    self.x =  random.uniform(-map_size, map_size)
    self.y = random.uniform(-map_size, map_size)

  def distance(self, other_location):
    dx = other_location.x - self.x
    dy = other_location.y - self.y
    return (dx**2+dy**2)**0.5

In [60]:
class Person:
  
  def __init__(self, world):

    self.world = world
    self.gender = random.choices(['M', 'F'])
    self.first_name = fake.first_name_female() if self.gender == 'F' else fake.first_name_male()
    self.last_name = fake.last_name()
    self.birthday = self.world.date
    self.deathday = None
    self.location = Location()
    self.velocity = 10
    self.energy = 100

  def __str__(self):
    return f"{self.first_name} {self.last_name} at {self.location.x},{self.location.y} \t energy: {self.energy}"
  
  def choose_action(self):
    
    for food in self.world.foods:
      if self.location.distance(food.location) < self.world.distance_can_eat:
        return 'eat', (self, food)

    day_to_death = self.energy / self.world.hunger_penalty
    dx = 0
    dy = 0
    for food in self.world.foods:
      if self.location.distance(food.location)/self.velocity < day_to_death:
        dx = food.location.x - self.location.x
        dy = food.location.y - self.location.y
        length = (dx**2+dy**2)**0.5
        dx_norm = (dx / length) * (self.velocity ** 0.5)
        dy_norm = (dy / length) * (self.velocity ** 0.5)
        if dx**2 + dy**2 < dx_norm**2 + dy_norm**2:
          dir_x, dir_y = dx, dy
        else:
          dir_x, dir_y= dx_norm, dy_norm
        break
    return 'move', (self, dx, dy)

In [61]:
class Food:

  def __init__(self, world):
    self.world = world
    self.location = Location()
    self.produce_date = self.world.date

In [67]:
class World:

  def __init__(self, name):
    self.name = name
    self.persons = []
    self.dead_persons = []
    self.date = datetime.datetime(2018, 6, 1)
    self.foods = []
    self.hunger_penalty = 20
    self.distance_can_eat = 5

  def simulate_day(self):
    self.date += datetime.timedelta(days=1)

    new_person = Person(world=self)
    self.persons.append(new_person)

    for i in range(10):
      self.foods.append(Food(self))
    self.foods[:] = filterfalse(lambda f: (world.date - f.produce_date).days > 5 , self.foods)

    for person in self.persons[:]:
      self.simulate_person(person)
  

  def simulate_person(self, person):
    person.energy -= self.hunger_penalty
    action, args = person.choose_action()
    
    if action == 'eat':
      food = args[1]
      self.foods.remove(food)
      person.energy = 100
    elif action == 'move':
      dx, dy = args[1], args[2]
      person.location.x += dx
      person.location.y += dy

    if person.energy == 0 :
      self.persons.remove(person)
      self.dead_persons.append(person)
      person.deathday = self.date

  def simulate(self, until=100):
    for i in range(until):
      self.simulate_day()
      #print(len(self.foods))
      print("persons ",len(self.persons), len(self.dead_persons))
      


In [68]:
world = World('mahdi_world')
world.simulate()

persons  1 0
persons  2 0
persons  3 0
persons  4 0
persons  5 0
persons  6 0
persons  7 0
persons  8 0
persons  9 0
persons  10 0
persons  10 1
persons  10 2
persons  10 3
persons  10 4
persons  10 5
persons  11 5
persons  11 6
persons  11 7
persons  12 7
persons  12 8
persons  13 8
persons  14 8
persons  15 8
persons  15 9
persons  16 9
persons  17 9
persons  16 11
persons  13 15
persons  14 15
persons  15 15
persons  16 15
persons  17 15
persons  18 15
persons  15 19
persons  15 20
persons  14 22
persons  14 23
persons  15 23
persons  14 25
persons  13 27
persons  14 27
persons  15 27
persons  16 27
persons  16 28
persons  17 28
persons  18 28
persons  17 30
persons  18 30
persons  17 32
persons  16 34
persons  16 35
persons  17 35
persons  17 36
persons  18 36
persons  17 38
persons  18 38
persons  19 38
persons  20 38
persons  21 38
persons  22 38
persons  23 38
persons  24 38
persons  23 40
persons  24 40
persons  24 41
persons  22 44
persons  22 45
persons  22 46
persons  21 48
